In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

# LlavaMetaModel 
from llava_arch import LlavaMetaModel
meta_llava = LlavaMetaModel(config)

# LlavaForCausalLM 


On MacOS, decord package is not compatible with new python version.
Loading vision tower: openai/clip-vit-base-patch32


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# from language_model.llava_llama import LlavaLlamaForCausalLM

from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

In [23]:
from PIL import Image
image = Image.open("multimodal_encoder/CLIP.png")
images = [image, image]

# Vision Tower: Preprocess & forward
processed_inputs = vision_tower.image_processor(images, return_tensors="pt")['pixel_values']
image_features = vision_tower(processed_inputs)

# Vision Projector: forward
image_features = vision_projector(image_features)


In [20]:
processed_inputs.shape

torch.Size([2, 3, 224, 224])

In [21]:
image_features.shape

torch.Size([2, 49, 768])

In [1]:
from builder import load_pretrained_model

On MacOS, decord package is not compatible with new python version.


In [1]:
from builder import initialize_llava_model
initialize_llava_model()

On MacOS, decord package is not compatible with new python version.
